In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage.measure import compare_psnr,compare_mse
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim as ssim
from random import shuffle
import math
from keras.models import model_from_json, load_model
from keras import optimizers
from keras.layers import *
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
import pickle

from utilities import *
%matplotlib inline


Using TensorFlow backend.


In [2]:
def readcifar(path) : 
    with open(path, 'rb') as f:
        train_set= pickle.load(f)
    return train_set["data"]

In [3]:
raw_train_set1 = readcifar('data/cifar-10-batches-py/data_batch_1')
raw_train_set2 = readcifar('data/cifar-10-batches-py/data_batch_2')
raw_train_set3 = readcifar('data/cifar-10-batches-py/data_batch_3')
raw_train_set4 = readcifar('data/cifar-10-batches-py/data_batch_4')
raw_train_set5 = readcifar('data/cifar-10-batches-py/data_batch_5')

In [4]:
raw_train_set = np.concatenate((raw_train_set1,raw_train_set2,raw_train_set3,raw_train_set4,raw_train_set5))

In [5]:
raw_train_set.shape

(50000, 3072)

In [6]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [7]:
def preprocess(images) :
    images = images.reshape(images.shape[0], 3, 32, 32).transpose(0,2,3,1).astype("uint8")
    print images.shape
    images = rgb2gray(images)
    print images.shape
    images = np.reshape(images, (images.shape[0],32,32))
    return images

In [8]:
train_set = preprocess(raw_train_set)

(50000, 32, 32, 3)
(50000, 32, 32)


In [9]:
train_set.shape

(50000, 32, 32)

In [10]:
for i in range(100):
    cv2.imwrite("./data/cifar-bw/image"+str(i)+".png",train_set[i])

In [4]:
#Greyscale noise
def add_gaussian_noise(images,mean,sigma,proportion):
    x_train_noisy=[]
    for i in range(images.shape[0]):
        img = images[i]
        noise_img,_ = add_greyscale_noise(img,mean,sigma)
        x_train_noisy.append(noise_img)
    return np.array(x_train_noisy)


def convert_to_multichannel(test_img):
    grey_noisy = np.zeros((test_img.shape[0],test_img.shape[1],test_img.shape[2],3),dtype=np.uint8)
    grey_noisy[:,:,:,0] = test_img
    grey_noisy[:,:,:,1] = test_img
    grey_noisy[:,:,:,2] = test_img
    
    return grey_noisy
#Adding noise to images
def add_noise(images):
    #x_train = np.reshape(x_train, (len(x_train), 64*64))  # adapt this if using `channels_first` image data format
    #x_test = np.reshape(x_test, (len(x_test), 64*64))  # adapt this if using `channels_first` image data format
    batch = images.shape[0]//5;
    noise1 = add_gaussian_noise(images[0:batch],0,15,1)
    noise2 = add_gaussian_noise(images[batch:2*batch],0,20,1)
    noise3 = add_gaussian_noise(images[2*batch:3*batch],0,25,1)
    noise4 = add_gaussian_noise(images[3*batch:4*batch],0,30,1)
    noise5 = add_gaussian_noise(images[4*batch:],0,35,1)
    
    noisy_set = []
    for data in [noise1,noise2,noise3,noise4,noise5]:
        for image in data:
            noisy_set.append(image)
    
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def pair_shuffle(images,noisy_set):
    image_pair = []
    for i in range(images.shape[0]):
        image_pair.append((images[i],noisy_set[i]))
    shuffle(image_pair)
    
    ground_truth=[]
    noisy_images = []
    for i in range(images.shape[0]):
        ground_truth.append(image_pair[i][0])
        noisy_images.append(image_pair[i][1])
    return np.array(ground_truth), np.array(noisy_images)

In [22]:
noisy_set = add_noise(train_set)
for i in range(100):
    cv2.imwrite("./data/noisy-cifar-bw/image"+str(i)+".png",noisy_set[-i])

In [23]:
#Shuffling and adding noise to the dataset
ground_truth,noisy_images = pair_shuffle(convert_to_multichannel(train_set),noisy_set)

In [26]:
#Split into training and cross validation and normalizing
train_size = int(ground_truth.shape[0]*0.75)
x_train = ground_truth[0:train_size]/255.
x_train_noisy = noisy_images[0:train_size]/255.
x_test = ground_truth[train_size:]/255.
x_test_noisy = noisy_images[train_size:]/255.
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(37500, 32, 32, 3)
(12500, 32, 32, 3)


In [27]:
lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('./models/simple_cnn_autoencoder.csv')
early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/simple_cnn_autoencoder.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [34]:
x_train[0].shape


In [39]:
#Defining the model

def get_simple_cnn_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=(None,None,3))  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    
        optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        
        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='Adam', loss='mean_squared_error',metrics = ['accuracy','mean_squared_error'])

    print (autoencoder.summary())
    return autoencoder

In [40]:
#Training the model
autoencoder = get_simple_cnn_autoencoder_model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, None, None, 64)    1792      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, None, None, 64)    0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_9 (UpSampling2 (None, None, None, 64)    0         
__________

In [41]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

Train on 37500 samples, validate on 12500 samples
Epoch 1/50
37500/37500 [==============================] - 99s - loss: 0.0148 - acc: 0.3482 - mean_squared_error: 0.0148 - val_loss: 0.0122 - val_acc: 0.3483 - val_mean_squared_error: 0.0122
Epoch 2/50
37500/37500 [==============================] - 98s - loss: 0.0121 - acc: 0.3403 - mean_squared_error: 0.0121 - val_loss: 0.0116 - val_acc: 0.3303 - val_mean_squared_error: 0.0116
Epoch 3/50
37500/37500 [==============================] - 98s - loss: 0.0116 - acc: 0.3370 - mean_squared_error: 0.0116 - val_loss: 0.0132 - val_acc: 0.2742 - val_mean_squared_error: 0.0132
Epoch 4/50
37500/37500 [==============================] - 98s - loss: 0.0111 - acc: 0.3404 - mean_squared_error: 0.0111 - val_loss: 0.0107 - val_acc: 0.3467 - val_mean_squared_error: 0.0107
Epoch 5/50
37500/37500 [==============================] - 98s - loss: 0.0108 - acc: 0.3427 - mean_squared_error: 0.0108 - val_loss: 0.0107 - val_acc: 0.3442 - val_mean_squared_error: 0.0107


37500/37500 [==============================] - 95s - loss: 0.0095 - acc: 0.4177 - mean_squared_error: 0.0095 - val_loss: 0.0098 - val_acc: 0.4248 - val_mean_squared_error: 0.0098
Epoch 42/50
37500/37500 [==============================] - 95s - loss: 0.0095 - acc: 0.4200 - mean_squared_error: 0.0095 - val_loss: 0.0100 - val_acc: 0.4206 - val_mean_squared_error: 0.0100
Epoch 43/50
37500/37500 [==============================] - 95s - loss: 0.0095 - acc: 0.4225 - mean_squared_error: 0.0095 - val_loss: 0.0099 - val_acc: 0.4124 - val_mean_squared_error: 0.0099
Epoch 44/50
37500/37500 [==============================] - 95s - loss: 0.0096 - acc: 0.4259 - mean_squared_error: 0.0096 - val_loss: 0.0098 - val_acc: 0.4356 - val_mean_squared_error: 0.0098
Epoch 45/50
37500/37500 [==============================] - 95s - loss: 0.0095 - acc: 0.4266 - mean_squared_error: 0.0095 - val_loss: 0.0106 - val_acc: 0.4255 - val_mean_squared_error: 0.0106
Epoch 46/50
37500/37500 [==============================] 

In [7]:
#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

In [117]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x: x[0]*x[1])([inputLayer,gateFactor])
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return gateFactor,fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(None,None,3))  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        gf1,fg1,c1 = get_gated_connections(0.1,x1)

        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        gf2,fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        gf3,fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        gf4,fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(3, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model([input_img,gf1,gf2,gf3,gf4],layers)
        sym_autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy','mean_squared_error'])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [118]:
sym_autoencoder = get_cnn_dsc_architecture()

Model created
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_16 (Conv2D)               (None, None, None, 64 3136        input_4[0][0]                    
____________________________________________________________________________________________________
input_5 (InputLayer)             (1,)                  0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, None, None, 64 0           conv2d_16[0][0]                  
                                                                   input_5[0]

In [119]:
model_checkpoint1 = ModelCheckpoint('./models/gated_cnn_autoencoder.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [121]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 37500 samples, validate on 12500 samples
Epoch 1/50
37500/37500 [==============================] - 453s - loss: 0.0095 - acc: 0.6346 - mean_squared_error: 0.0095 - val_loss: 0.0047 - val_acc: 0.6947 - val_mean_squared_error: 0.0047
Epoch 2/50
37500/37500 [==============================] - 450s - loss: 0.0041 - acc: 0.7089 - mean_squared_error: 0.0041 - val_loss: 0.0037 - val_acc: 0.7193 - val_mean_squared_error: 0.0037
Epoch 3/50
37500/37500 [==============================] - 450s - loss: 0.0035 - acc: 0.7274 - mean_squared_error: 0.0035 - val_loss: 0.0034 - val_acc: 0.7318 - val_mean_squared_error: 0.0034
Epoch 4/50
37500/37500 [==============================] - 450s - loss: 0.0033 - acc: 0.7366 - mean_squared_error: 0.0033 - val_loss: 0.0032 - val_acc: 0.7400 - val_mean_squared_error: 0.0032
Epoch 5/50
37500/37500 [==============================] - 450s - loss: 0.0031 - acc: 0.7428 - mean_squared_error: 0.0031 - val_loss: 0.0031 - val_acc: 0.7454 - val_mean_squared_error: 0.

37500/37500 [==============================] - 448s - loss: 0.0023 - acc: 0.7731 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_acc: 0.7741 - val_mean_squared_error: 0.0023
Epoch 41/50
37500/37500 [==============================] - 448s - loss: 0.0023 - acc: 0.7732 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_acc: 0.7738 - val_mean_squared_error: 0.0023
Epoch 42/50
37500/37500 [==============================] - 448s - loss: 0.0023 - acc: 0.7735 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_acc: 0.7752 - val_mean_squared_error: 0.0023
Epoch 43/50
37500/37500 [==============================] - 448s - loss: 0.0023 - acc: 0.7736 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_acc: 0.7743 - val_mean_squared_error: 0.0023
Epoch 44/50
37500/37500 [==============================] - 448s - loss: 0.0023 - acc: 0.7738 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_acc: 0.7743 - val_mean_squared_error: 0.0023
Epoch 45/50
37500/37500 [===========================

In [16]:

def get_psnr(imageA,imageB):
    maxI = 255.
    try:
        return 20*math.log10(maxI) - 10*math.log10(compare_mse(imageA.flatten(),imageB.flatten()))
    except:
        return 20*math.log10(maxI)

def get_psnr_result(x_test, out):
    psnr_sum = 0
    for i in range(out.shape[0]):
        psnr_sum += compare_psnr(x_test[i].reshape(32,32,1),out[i].reshape(32,32,1),data_range=255)
        
    return 1.0*psnr_sum/out.shape[0];

def get_ssim_result(originalSet,noisySet):
    ssim_sum = 0
    originalSet = originalSet.reshape(originalSet.shape[0],32,32,1)
    noisySet = noisySet.reshape(noisySet.shape[0],32,32,1)
    for i in range(originalSet.shape[0]):
        ssim_sum += ssim(originalSet[i], noisySet[i],data_range=originalSet[i].max() - noisySet[i].min(), multichannel=True)
    return 1.0*ssim_sum/originalSet.shape[0]

In [17]:
def bm3d_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32)
    denoised = []
    count = 1
    for i in range(noisy_image.shape[0]):
        Basic_img = bm3d.BM3D_1st_step(noisy_image[i])
        Final_img = bm3d.BM3D_2nd_step(Basic_img, noisy_image[i])
        denoised.append(Final_img)
        if (count%10 == 0):
            print (str(count)+ "images denoised")
        count+=1
        
    return np.array(denoised)

def nlm_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,1)
    denoised = []
    count = 1
    
    for image in noisy_image:
        denoised_image = denoise_nl_means(image, 7, 11, 0.5,multichannel = False)
        denoised.append(denoised_image)
        if(count%100 == 0) :
            print(str(count)+" images denoised")
        count+=1
    return np.array(denoised)

In [134]:
print (test_images.shape,out_autoencoder.shape)
print (compare_mse(test_images[0].flatten(),(out_autoencoder[0]*255.).flatten()))
get_psnr_result(out_autoencoder*255.,test_images)

((10000, 32, 32, 1), (10000, 32, 32, 1))
458.092258419


22.066576979897317

In [39]:
print (test_images.shape,out_sym_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_sym_autoencoder[0].flatten()))
get_psnr_result(out_sym_autoencoder,test_images)

NameError: name 'out_sym_autoencoder' is not defined

In [39]:
import bm3d

In [40]:
noisy_test_images.shape
bm3d_out = bm3d_denoise(noisy_test_images)

error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/core/src/dxt.cpp:2330: error: (-213) Odd-size DCT's are not implemented in function dct


In [ ]:
bm3d_out_norm = bm3d_out.astype('float64')/255.0

In [ ]:
get_psnr_result(bm3d_out_norm,test_images)

In [166]:
print(noisy_test_images.shape)
nlm_out = nlm_denoise(noisy_test_images*255.)
nlm_out = nlm_out.astype('float64')

(10000, 32, 32, 1)


KeyboardInterrupt: 

In [157]:
print nlm_out.shape,test_images.shape
get_psnr_result(nlm_out,test_images)

(10000, 32, 32) (10000, 32, 32, 1)


18.414818545893922

In [136]:
get_ssim_result(test_images,out_autoencoder*255.)

0.69608629344603479

In [66]:
get_ssim_result(test_images,out_sym_autoencoder)

0.718734871276698

In [ ]:
get_ssim_result(test_images,bm3d_out_norm)

In [69]:
get_ssim_result(test_images,nlm_out)

0.49438484841716263

In [57]:
model = load_model("./models/simple_cnn_autoencoder.hdf5")

In [21]:
bwmodel= model.predict(noisy_test_images,verbose=1)

10000/10000 [==============================] - 7s     


In [24]:
print (test_images.shape,bwmodel.shape)
get_psnr_result(bwmodel,test_images)

((10000, 32, 32, 1), (10000, 32, 32, 1))


16.773690910050014

In [18]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append((img,filename))
    return np.array(images)
        

In [19]:
standard_test_images = read_images_from_folder('./data/standard_test_images/gd/')

OSError: [Errno 2] No such file or directory: './data/standard_test_images/gd/'

In [140]:
standard_test_images.shape

(12, 2)

In [141]:
standard_test_images[0]

array([ array([[156, 157, 160, ..., 152, 152, 152],
       [156, 157, 159, ..., 152, 152, 152],
       [158, 157, 156, ..., 152, 152, 152],
       ..., 
       [121, 123, 126, ..., 121, 113, 111],
       [121, 123, 126, ..., 121, 113, 111],
       [121, 123, 126, ..., 121, 113, 111]], dtype=uint8),
       'cameraman.tif'], dtype=object)

In [154]:
std_test_images = []
image_names = []
for img in standard_test_images:
    std_test_images.append(img[0])
    image_names.append(img[1])
std_test_images = np.array(test_images)

In [143]:
noisy_std_test_images = add_noise(test_images)

In [144]:
for index in range(noisy_std_test_images.shape[0]):
    cv2.imwrite('./data/standard_test_images/noise/'+image_names[index],noisy_std_test_images[index])

In [14]:
def denoise_std_test_images(images,model,stride):
    denoised_images = []
    for index in range(images.shape[0]):
        d_image = denoise_by_patches(images[index],model,stride)
        denoised_images.append(d_image)
        print compare_psnr(d_image.astype('int64'),std_test_images[index].astype('int64'),data_range=255)
        cv2.imwrite('./data/standard_test_images/denoise/'+image_names[index],d_image)
        print "image " + str(index)+ " denoised"

In [231]:
denoise_std_test_images(noisy_std_test_images,model,1)

KeyboardInterrupt: 

In [196]:
nlm_camera = cv2.fastNlMeansDenoising(noisy_std_test_images[0].astype('uint8'),None,25,7 ,21)
cv2.imwrite("nlm.png",nlm_camera)
print compare_psnr(nlm_camera,std_test_images[0])

30.1276904426


In [13]:
autoencoder = load_model('./models/cluster/simple_cnn_autoencoder.hdf5')
sym_autoencoder = load_model('./models/gated_cnn_autoencoder.hdf5')

In [6]:
def get_ssim(originalSet,noisySet):
    return ssim(originalSet, noisySet,data_range=originalSet.max() - noisySet.min(), multichannel=True)

In [7]:
def scale_image(img,vmin,vmax):
    umin = img.min()
    umax = img.max()
    
    sf = 1.0*(vmax - vmin)/(umax - umin)
    scaled_image = vmin + 1.0*(img - umin)*sf
    return scaled_image

In [8]:
def add_multichannel_gaussian(test_img,noise_std):
    noise = np.random.normal(0,noise_std, (test_img.shape[0],test_img.shape[1])).astype('uint8')
    noisy_test = np.ones((test_img.shape[0],test_img.shape[1],3),dtype=np.uint8)
    noisy_test[:,:,0] = test_img[:,:,0] + noise
    noisy_test[:,:,1] = test_img[:,:,1] + noise
    noisy_test[:,:,2] = test_img[:,:,2] +noise
    noisy_test = np.clip(noisy_test,0.,255.)
    
    return noisy_test

def add_greyscale_noise(test_img,mean,noise_std):
    noise = np.random.normal(mean,noise_std, (test_img.shape[0],test_img.shape[1])).astype('float64')
    test_img = test_img + noise
  
    grey_test = scale_image(test_img,0,255)
    
    grey_noisy = np.zeros((test_img.shape[0],test_img.shape[1],3),dtype=np.uint8)
    grey_noisy[:,:,0] = grey_test
    grey_noisy[:,:,1] = grey_test
    grey_noisy[:,:,2] = grey_test
    
    
    
    return grey_noisy,grey_test

In [14]:
#plt.imshow(out_autoencoder[0])

def predict_save(model,image_path):
    
    noise_std = 30
    test_img = cv2.imread(image_path,0)
    cv2.imwrite("gt.png",test_img)
    test_img = np.array(test_img)
    
    #Adding noise
    #noisy_test is grayscale in multichannel and noisy_test_SC is grayscale in singlechannel
    noisy_test,noisy_test_SC = add_greyscale_noise(test_img,0,noise_std)
    cv2.imwrite("noisy.png",noisy_test_SC)
    print compare_psnr(test_img.astype('uint8'),noisy_test_SC.astype('uint8')), get_ssim(test_img.astype('uint8'),noisy_test_SC.astype('uint8'))
    
    
    #NLM Denoise
    #nlm_out = cv2.fastNlMeansDenoising(noisy_test.astype('uint8'),noise_std,7,21)
    #print nlm_out.shape,test_img.shape
    #cv2.imwrite("nlm_denoise.png",nlm_out)
    #print compare_psnr(nlm_out.astype('uint8'),test_img.astype('uint8')), get_ssim(nlm_out.astype('uint8'),test_img.astype('uint8'))
    

    
    noisy_test = noisy_test.reshape(1,noisy_test.shape[0],noisy_test.shape[1],noisy_test.shape[2])
    noisy_test = noisy_test/255.0
    
    out = model.predict(noisy_test,verbose=1)
    
    output_img = (out[0][:,:,0]*255.).astype('uint8');
    #output_img = (out[0]*255.).astype('uint8');
    #print out[0][:,:,1]*255.-out[0][:,:,2]*255.
    cv2.imwrite("denoised.png",output_img)
    #print output_img[:,:,0].shape,test_img.shape
    print compare_psnr(output_img,test_img.astype('uint8')), get_ssim(output_img,test_img.astype('uint8'))
    

In [17]:
predict_save(autoencoder,'./data/standard_test_images/gt/cameraman.tif')

17.7711530511 0.269747860059
1/1 [==============================] - 0s
21.7866181342 0.720469256745


In [227]:
help(cv2.fastNlMeansDenoising)

Help on built-in function fastNlMeansDenoising in module cv2:

fastNlMeansDenoising(...)
    fastNlMeansDenoising(src[, dst[, h[, templateWindowSize[, searchWindowSize]]]]) -> dst

